In [3]:
from osgeo import gdal
import rasterio
import json

#### Preprocessing functions

In [1]:
# Function for getting MetaData of the file
def get_metadata(src,name):
    name = name.split(".")[0]
    height = src.height
    width = src.width
    band_count = src.count
    driver = src.driver
    dtype = src.meta['dtype']
    nodata = src.meta['nodata']
    affine = {
                "a":src.transform.a,
                "b": src.transform.b,
                "c": src.transform.c,
                "d": src.transform.d,
                "e": src.transform.e,
                "f": src.transform.f
            }

    #getting crs
    crs_temp = src.crs.to_string()
    
    #getting bounds
    bounds = {
        "bottom": src.bounds.bottom,
        "top":src.bounds.top,
        "left":src.bounds.left,
        "right":src.bounds.right
    }
    metadata = {
        "height":height,
        "width":width,
        "crs":crs_temp,
        "bounds":json.dumps(bounds),
        "band_count":band_count,
        "driver":driver,
        "dtype":dtype,
        "nodata":nodata,
        "name":name,
        "affine":json.dumps(affine),

    }
    return metadata


In [2]:
def check_in_correct_crs(metadata):
    crs = metadata['crs']

    if crs.lower() == "epsg:4326":
        return True
    else:
        return False

In [ ]:
#Do reprojection of the image to EPSG:4326 crs format and return the new file path
def do_reprojection(input_path):
    dst_crs = 'EPSG:4326'
    if not os.path.exists(os.path.join(os.path.abspath(os.getcwd()),"temp")):
        os.mkdir(os.path.join(os.path.abspath(os.getcwd()),"temp"))

    output_path = os.path.join(os.path.join(os.path.abspath(os.getcwd()),"temp"),os.path.basename(input_path))
    
    filename = input_path
    input_raster = gdal.Open(filename)
    output_raster = output_path
    gdal.Warp(output_raster,input_raster,dstSRS=dst_crs)

    return output_path
